# Cellucid Jupyter Hooks + Sessions (HE developmental example)

This notebook is a **hands-on test** for the Jupyter integration:
- embedding + connectivity
- hooks (`on_ready`, `on_selection`, `on_hover`, `on_click`)
- `viewer.state` snapshots
- no-download session capture (`viewer.get_session_bundle`)
- apply session → `AnnData`

Dataset: `cellucid-python/data/experiments/he_developmental_complete_with_3d_umap.h5ad`


In [ ]:
from __future__ import annotations

from pathlib import Path

import anndata

from cellucid import show_anndata


In [ ]:
# Locate the dataset (works whether your CWD is the repo root or this examples folder).
candidates = [
    Path('data/experiments/he_developmental_complete_with_3d_umap.h5ad'),
    Path('../../../../data/experiments/he_developmental_complete_with_3d_umap.h5ad'),
]

DATASET = next((p for p in candidates if p.exists()), None)
if DATASET is None:
    raise FileNotFoundError('Could not find the HE developmental dataset in expected locations')

DATASET = DATASET.resolve()
DATASET


In [ ]:
# Load AnnData in backed mode (keeps memory usage lower).
adata = anndata.read_h5ad(DATASET, backed='r')
adata


In [ ]:
# Launch the viewer.
# In notebooks, this auto-displays an iframe and starts a background server.
viewer = show_anndata(DATASET, height=650)
viewer


In [ ]:
# Hook examples (UI → Python).
@viewer.on_ready
def _on_ready(event):
    print('READY:', event)

@viewer.on_click
def _on_click(event):
    print('CLICK:', event.get('cell'), 'button=', event.get('button'))

@viewer.on_hover
def _on_hover(event):
    # Hover is frequent; only print when entering a cell.
    cell = event.get('cell')
    if cell is not None:
        print('HOVER:', cell)

@viewer.on_selection
def _on_selection(event):
    # Note: in notebooks, selection is emitted when you CONFIRM a selection into a highlight group.
    cells = event.get('cells') or []
    print('SELECTION:', len(cells), 'source=', event.get('source'))


## Try it

1. Wait for the viewer to finish loading.
2. Make a selection in the UI and **confirm** it into a highlight group.
3. Come back and run the next cells to pull state and capture a session bundle.


In [ ]:
viewer.wait_for_ready(timeout=120)
viewer.state


In [ ]:
# Capture the current session as a Python object (no manual download).
bundle = viewer.get_session_bundle(timeout=120)
len(bundle.list_chunk_ids()), bundle.dataset_fingerprint


In [ ]:
# Apply the session bundle back onto AnnData.
# This currently materializes highlight memberships and categorical user-defined fields.
adata2, summary = bundle.apply_to_anndata(adata, inplace=False, return_summary=True)
summary


In [ ]:
# Inspect the newly added columns (names depend on highlight group IDs and field keys).
[c for c in adata2.obs.columns if c.startswith('cellucid_')]


In [ ]:
# If anything feels off, this report is the fastest way to diagnose connectivity.
viewer.debug_connection()


In [ ]:
# Cleanup: stops the server and freezes the displayed view (non-interactive, visually unchanged).
# viewer.stop()
